<a href="https://colab.research.google.com/github/sbooeshaghi/BMGP_2020/blob/main/analysis/notebooks/10xFB/10xFB_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!date

Wed Jun 16 21:28:18 UTC 2021


# Install packages and download data

In [2]:
import sys
COLAB = "google.colab" in sys.modules


In [4]:
if COLAB:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 678, done.
remote: Counting objects: 100% (382/382), done.
remote: Compressing objects: 100% (338/338), done.
remote: Total 678 (delta 149), reused 173 (delta 41), pack-reused 296
Receiving objects: 100% (678/678), 469.79 MiB | 18.93 MiB/s, done.
Resolving deltas: 100% (240/240), done.
Checking out files: 100% (144/144), done.


In [ ]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.0 \
    anndata==0.7.6

     |████████████████████████████████| 59.1MB 71kB/s 
     |████████████████████████████████| 133kB 34.3MB/s 
     |████████████████████████████████| 10.3MB 21.5MB/s 
     |████████████████████████████████| 13.2MB 154kB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 112kB 52.3MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 1.2MB 32.2MB/s 


In [4]:
if COLAB:
    # download cellranger (30sec)
    !wget -O cellranger-6.0.1.tar.gz \
    -q --show-progress --progress=bar:force  \
    "https://caltech.box.com/shared/static/i9dwsh2h6kojewjk967sjffdkunsk5tj.gz" \
    2>&1
    
    !tar -xf cellranger-6.0.1.tar.gz

    
    # download cellranger reference (5min)
    !wget \
    -q --show-progress --progress=bar:force  \
    https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz \
    2>&1
    
    !tar -xf refdata-gex-GRCh38-2020-A.tar.gz


cellranger-6.0.1.ta 100%[===================>]   1.01G  20.3MB/s    in 46s     
refdata-gex-GRCh38- 100%[===================>]  10.66G  38.5MB/s    in 4m 47s  


In [5]:
%%bash
source cellranger-6.0.1/sourceme.bash

In [ ]:
if COLAB:
    # download the relevant data
    !wget -O 10xFB_5k_pbmc_v3_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/z28rkegt9rdrmjlly9ashv7jqtghztjp.gz \
    2>&1
    !tar -xf 10xFB_5k_pbmc_v3_fastqs.tar.gz

10xFB_5k_pbmc_v3_fa 100%[===================>]   2.60G  18.1MB/s    in 2m 13s  


# Preprocess with kb

In [ ]:
%%time
!FB="BMGP_2020/references/10xFB/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/10xFB/kite/features.idx" && \
F2B="BMGP_2020/references/10xFB/kite/f2b.txt" && \
FASTA="BMGP_2020/references/10xFB/kite/features.fa" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-06-14 22:18:51,938]    INFO Generating mismatch FASTA at BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.fa
[2021-06-14 22:18:51,969]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/10xFB_5k_pbmc_v3/kite/f2b.txt
[2021-06-14 22:18:51,976]    INFO Skipping kallisto index because BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 27 ms, sys: 7.25 ms, total: 34.2 ms
Wall time: 3.85 s


In [5]:
%%bash
FB="BMGP_2020/references/10xFB/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/10xFB/kite/features.idx" && \
F2B="BMGP_2020/references/10xFB/kite/f2b.txt" && \
FASTA="BMGP_2020/references/10xFB/kite/features.fa" && \
OUT="kite_out" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
/usr/bin/time --output=memtime_0.log -v kb count \
-i $INDEX \
-g $F2B \
-x 10xv3 \
-o $OUT \
--workflow kite:10xFB \
--h5ad \
--filter bustools \
$FASTQS

[2021-06-30 15:18:43,857]    INFO [count] Using index BMGP_2020/references/10xFB/kite/features.idx to generate BUS file to kite_out from
[2021-06-30 15:18:43,858]    INFO [count]         fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R1_001.fastq.gz
[2021-06-30 15:18:43,858]    INFO [count]         fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R2_001.fastq.gz
[2021-06-30 15:18:43,858]    INFO [count]         fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R1_001.fastq.gz
[2021-06-30 15:18:43,858]    INFO [count]         fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R2_001.fastq.gz
[2021-06-30 15:18:43,868]   ERROR [count] kallisto 0.46.2
Generates BUS files for single-cell sequencing

Usage: kallisto bus [arguments] FASTQ-files

Required arguments:
-i, --index=STRING            Filename for the kallisto index to be used for
pseudoalignment
-o, --output-dir=STRING       Directory to write output to
-x, --technology=STRING       Single-cell technology used

Optional arguments:
-l, --list               

In [ ]:
!cat kite_out/run_info.json

{
	"n_targets": 0,
	"n_bootstraps": 0,
	"n_processed": 39096673,
	"n_pseudoaligned": 37262835,
	"n_unique": 37262835,
	"p_pseudoaligned": 95.3,
	"p_unique": 95.3,
	"kallisto_version": "0.46.2",
	"index_version": 0,
	"start_time": "Mon Jun 14 22:18:53 2021",
	"call": "/usr/local/lib/python3.7/dist-packages/kb_python/bins/linux/kallisto/kallisto bus -i BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.idx -o kite_out -x 10xv3 -t 8 fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R1_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R2_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R1_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R2_001.fastq.gz"
}


# Preprocess with cellranger

In [ ]:
!printf "fastqs,sample,library_type\n$PWD/fastqs,5k_pbmc_protein_v3_antibody,Antibody Capture" > BMGP_2020/references/10xFB_5k_pbmc_v3/5k_pbmc_protein_v3_libraries.csv

In [7]:
!./cellranger-6.0.1/cellranger

cellranger cellranger-6.0.1
Process 10x Genomics Gene Expression, Feature Barcode, and Immune Profiling data

USAGE:
    cellranger <SUBCOMMAND>

FLAGS:
    -h, --help       Prints help information
    -V, --version    Prints version information

SUBCOMMANDS:
    count               Count gene expression (targeted or whole-transcriptome) and/or feature barcode reads from a
                        single sample and GEM well
    multi               Analyze multiplexed data or combined gene expression/immune profiling/feature barcode data
    vdj                 Assembles single-cell VDJ receptor sequences from 10x Immune Profiling libraries
    aggr                Aggregate data from multiple Cell Ranger runs
    reanalyze           Re-run secondary analysis (dimensionality reduction, clustering, etc)
    targeted-compare    Analyze targeted enrichment performance by comparing a targeted sample to its cognate parent
                        WTA sample (used as input for targeted gene expr

In [ ]:
!LIB="BMGP_2020/references/10xFB/5k_pbmc_protein_v3_libraries.csv" && \
FR="BMGP_2020/references/10xFB/5k_pbmc_protein_v3_feature_ref.csv" && \
./cellranger-6.0.1/cellranger count \
--libraries $LIB \
--feature-ref $FR \
--id count \
--nosecondary \
--disable-ui \
--expect-cells 5000 \
--transcriptome refdata-gex-GRCh38-2020-A

Martian Runtime - v4.0.4
2021-04-27 13:52:55 [jobmngr] WARNING: configured to use 340GB of local memory, but only 263.3GB is currently available.
Serving UI at http://atlas:42761?auth=UW3XRrq4jVUqusPKpmLnu1rRB_JNV0tF_ZAOk3lxgYg

Running preflight checks (please wait)...
Checking feature definition file...
Checking sample info...
Checking FASTQ folder...
Checking reference...
Checking reference_path (/home/sina/projects/kite/BMGP_2020/analysis/notebooks/refdata-gex-GRCh38-2020-A) on atlas...
Checking optional arguments...
mrc: v4.0.4

mrp: v4.0.4

Anaconda: Python 3.7.6

numpy: 1.19.2

scipy: 1.1.0

pysam: 0.16.0.1

h5py: 2.8.0

pandas: 0.24.2

STAR: 2.7.2a

samtools: samtools 1.10
Using htslib 1.10.2
Copyright (C) 2019 Genome Research Ltd.

2021-04-27 13:54:08 [runtime] (ready)           ID.count.SC_RNA_COUNTER_CS.SC_MULTI_CORE.MULTI_CHEMISTRY_DETECTOR._GEM_WELL_CHEMISTRY_DETECTOR.DETECT_COUNT_CHEMISTRY
2021-04-27 13:54:08 [runtime] (run:local)       ID.count.SC_RNA_COUNTER_CS.SC_MULTI